# MÓDULO 8: INTRODUCCIÓN A LANGGRAPH
## SESIÓN 1: Soluciones Comentadas - Ingeniería de Agentes

**Formato de comentarios:**
- `# MEMORIA:` Explica cambios en el estado (post-it compartido)
- `# CAPACIDAD:` Describe la responsabilidad del nodo
- `# DECISIÓN:` Justifica la lógica del router (GPS)
- `# ARQUITECTURA:` Explica conexiones del grafo

Importamos las dependencias necesarias para construir agentes con LangGraph.

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

In [ ]:
# EJERCICIO 1: Definir la Memoria de Corto Plazo del Agente

### MEMORIA: El Estado es el "post-it" que el agente lleva consigo
### Cada campo representa información que necesita recordar durante su trabajo
class EstadoTareas(TypedDict):
    tareas: List[str]        # MEMORIA: Lista de tareas pendientes (backlog)
    tarea_actual: str        # MEMORIA: Qué tarea está procesando ahora (working memory)
    completadas: int         # MEMORIA: Contador de progreso (métrica de rendimiento)
    estado: str              # MEMORIA: Estado operacional del agente ("activo" | "inactivo")

# Prueba: Inicializar la memoria del agente
estado_prueba: EstadoTareas = {
    "tareas": ["Comprar pan", "Hacer ejercicio"],
    "tarea_actual": "",
    "completadas": 0,
    "estado": "activo"
}

print(f"Estado creado: {estado_prueba}")

---
## EJERCICIO 2: Crear una Capacidad Simple (Normalización de Texto)

In [ ]:
class EstadoTexto(TypedDict):
    texto_entrada: str
    texto_salida: str

# CAPACIDAD: Normalización de texto
# Este nodo tiene UNA responsabilidad: convertir texto a formato estándar
def nodo_transformar(state: EstadoTexto) -> EstadoTexto:
    """
    Capacidad: Normalización de texto.
    Lee el texto crudo y lo convierte a formato limpio y consistente.
    """
    # MEMORIA: Leer el input del post-it compartido
    texto = state["texto_entrada"]
    
    # CAPACIDAD: Aplicar transformación (minúsculas + trim)
    texto_procesado = texto.lower().strip()
    
    # MEMORIA: Escribir el resultado en el post-it (solo el campo modificado)
    return {"texto_salida": texto_procesado}

# Prueba
estado_test = {"texto_entrada": "  HOLA MUNDO  ", "texto_salida": ""}
resultado = nodo_transformar(estado_test)
print(f"Resultado: '{resultado['texto_salida']}'")
print("Esperado: 'hola mundo'")

---
## EJERCICIO 3: Pipeline de Procesamiento (Cadena de Capacidades)

In [ ]:
class EstadoNumero(TypedDict):
    numero: int

# CAPACIDAD 1: Validación de entrada
# Responsabilidad: Garantizar que el número es positivo
def nodo_validar(state: EstadoNumero) -> EstadoNumero:
    # MEMORIA: Leer número del estado
    numero = state["numero"]
    
    # CAPACIDAD: Convertir negativo a positivo si es necesario
    if numero < 0:
        numero = abs(numero)  # Valor absoluto
    
    # MEMORIA: Actualizar el número validado
    return {"numero": numero}

# CAPACIDAD 2: Multiplicación
# Responsabilidad: Duplicar el valor
def nodo_duplicar(state: EstadoNumero) -> EstadoNumero:
    # CAPACIDAD: Aplicar transformación matemática
    numero = state["numero"] * 2
    return {"numero": numero}

# CAPACIDAD 3: Suma
# Responsabilidad: Incrementar valor final
def nodo_sumar_diez(state: EstadoNumero) -> EstadoNumero:
    # CAPACIDAD: Aplicar offset
    numero = state["numero"] + 10
    return {"numero": numero}

# ARQUITECTURA: Construir pipeline lineal
workflow_numeros = StateGraph(EstadoNumero)

# ARQUITECTURA: Registrar las 3 capacidades en el grafo
workflow_numeros.add_node("validar", nodo_validar)
workflow_numeros.add_node("duplicar", nodo_duplicar)
workflow_numeros.add_node("sumar_diez", nodo_sumar_diez)

# ARQUITECTURA: Definir el punto de entrada (primera capacidad)
workflow_numeros.set_entry_point("validar")

# ARQUITECTURA: Conectar capacidades en secuencia (pipeline)
workflow_numeros.add_edge("validar", "duplicar")       # validar → duplicar
workflow_numeros.add_edge("duplicar", "sumar_diez")    # duplicar → sumar_diez
workflow_numeros.add_edge("sumar_diez", END)           # sumar_diez → fin

# ARQUITECTURA: Compilar el grafo (convertir diseño en agente ejecutable)
app_numeros = workflow_numeros.compile()

# Prueba con número negativo
estado_inicial = {"numero": -5}
resultado = app_numeros.invoke(estado_inicial)
print(f"Resultado: {resultado['numero']}")
print("Esperado: (|-5| * 2) + 10 = 20")

---
## EJERCICIO 4: Sistema de Decisión Dinámica (Router como GPS)

In [ ]:
class EstadoProducto(TypedDict):
    precio: float
    categoria: str
    precio_final: float

# CAPACIDAD: Clasificador de productos
# Responsabilidad: Determinar la categoría según el precio
def nodo_categorizar(state: EstadoProducto) -> EstadoProducto:
    # MEMORIA: Leer precio del estado
    precio = state["precio"]
    
    # CAPACIDAD: Aplicar lógica de clasificación
    if precio < 10:
        categoria = "barato"
    elif precio <= 50:
        categoria = "medio"
    else:
        categoria = "caro"
    
    # MEMORIA: Actualizar categoría en el estado
    return {"categoria": categoria}

# DECISIÓN: Router (GPS inteligente)
# Responsabilidad: Decidir qué capacidad de descuento aplicar
def router_precio(state: EstadoProducto) -> str:
    # DECISIÓN: Leer la categoría y elegir la ruta
    # Devuelve el NOMBRE del nodo destino (no el nodo mismo)
    return state["categoria"]  # "barato" | "medio" | "caro"

# CAPACIDAD: Descuento para productos baratos (5%)
def nodo_descuento_barato(state: EstadoProducto) -> EstadoProducto:
    # CAPACIDAD: Aplicar 5% de descuento
    precio_final = state["precio"] * 0.95
    return {"precio_final": precio_final}

# CAPACIDAD: Descuento para productos medios (10%)
def nodo_descuento_medio(state: EstadoProducto) -> EstadoProducto:
    # CAPACIDAD: Aplicar 10% de descuento
    precio_final = state["precio"] * 0.90
    return {"precio_final": precio_final}

# CAPACIDAD: Descuento para productos caros (15%)
def nodo_descuento_caro(state: EstadoProducto) -> EstadoProducto:
    # CAPACIDAD: Aplicar 15% de descuento
    precio_final = state["precio"] * 0.85
    return {"precio_final": precio_final}

# ARQUITECTURA: Construir grafo con bifurcación dinámica
workflow_producto = StateGraph(EstadoProducto)

# ARQUITECTURA: Registrar capacidades
workflow_producto.add_node("categorizar", nodo_categorizar)
workflow_producto.add_node("barato", nodo_descuento_barato)
workflow_producto.add_node("medio", nodo_descuento_medio)
workflow_producto.add_node("caro", nodo_descuento_caro)

# ARQUITECTURA: Punto de entrada
workflow_producto.set_entry_point("categorizar")

# ARQUITECTURA: Edge condicional (bifurcación con GPS)
# El router decide dinámicamente qué capacidad de descuento activar
workflow_producto.add_conditional_edges(
    "categorizar",           # Desde este nodo...
    router_precio,           # ...usa este GPS para decidir...
    {                        # ...entre estas rutas posibles:
        "barato": "barato",  # Si router devuelve "barato" → ir a nodo "barato"
        "medio": "medio",    # Si router devuelve "medio" → ir a nodo "medio"
        "caro": "caro"       # Si router devuelve "caro" → ir a nodo "caro"
    }
)

# ARQUITECTURA: Todas las rutas convergen en END
workflow_producto.add_edge("barato", END)
workflow_producto.add_edge("medio", END)
workflow_producto.add_edge("caro", END)

# ARQUITECTURA: Compilar
app_producto = workflow_producto.compile()

# Pruebas con diferentes precios
productos_test = [
    {"precio": 5.0, "categoria": "", "precio_final": 0.0},
    {"precio": 30.0, "categoria": "", "precio_final": 0.0},
    {"precio": 100.0, "categoria": "", "precio_final": 0.0}
]

for producto in productos_test:
    resultado = app_producto.invoke(producto)
    print(f"Precio original: ${resultado['precio']:.2f} → Categoría: {resultado['categoria']} → Precio final: ${resultado['precio_final']:.2f}")

In [ ]:
class EstadoProducto(TypedDict):
    precio: float
    categoria: str
    precio_final: float

# Solución: Nodo que categoriza según el precio
def nodo_categorizar(state: EstadoProducto) -> EstadoProducto:
    precio = state["precio"]
    
    if precio < 10:
        categoria = "barato"
    elif precio <= 50:
        categoria = "medio"
    else:
        categoria = "caro"
    
    return {"categoria": categoria}

# Solución: Router que decide según la categoría
def router_precio(state: EstadoProducto) -> str:
    return state["categoria"]  # Devuelve "barato", "medio" o "caro"

# Solución: Nodos de descuento
def nodo_descuento_barato(state: EstadoProducto) -> EstadoProducto:
    precio_final = state["precio"] * 0.95  # 5% descuento
    return {"precio_final": precio_final}

def nodo_descuento_medio(state: EstadoProducto) -> EstadoProducto:
    precio_final = state["precio"] * 0.90  # 10% descuento
    return {"precio_final": precio_final}

def nodo_descuento_caro(state: EstadoProducto) -> EstadoProducto:
    precio_final = state["precio"] * 0.85  # 15% descuento
    return {"precio_final": precio_final}

# Solución: Construir grafo con edges condicionales
workflow_producto = StateGraph(EstadoProducto)

# Añadir nodos
workflow_producto.add_node("categorizar", nodo_categorizar)
workflow_producto.add_node("barato", nodo_descuento_barato)
workflow_producto.add_node("medio", nodo_descuento_medio)
workflow_producto.add_node("caro", nodo_descuento_caro)

# Entry point
workflow_producto.set_entry_point("categorizar")

# Edge condicional desde categorizar
workflow_producto.add_conditional_edges(
    "categorizar",
    router_precio,
    {
        "barato": "barato",
        "medio": "medio",
        "caro": "caro"
    }
)

# Todos van a END
workflow_producto.add_edge("barato", END)
workflow_producto.add_edge("medio", END)
workflow_producto.add_edge("caro", END)

# Compilar
app_producto = workflow_producto.compile()

# Pruebas
productos_test = [
    {"precio": 5.0, "categoria": "", "precio_final": 0.0},
    {"precio": 30.0, "categoria": "", "precio_final": 0.0},
    {"precio": 100.0, "categoria": "", "precio_final": 0.0}
]

for producto in productos_test:
    resultado = app_producto.invoke(producto)
    print(f"Precio original: ${resultado['precio']:.2f} → Categoría: {resultado['categoria']} → Precio final: ${resultado['precio_final']:.2f}")

---
## EJERCICIO 5: Grafo de Validación de Email

In [ ]:
class EstadoEmail(TypedDict):
    email: str
    tiene_arroba: bool
    dominio_valido: bool
    resultado: str

# Solución: Nodo verificar arroba
def nodo_verificar_arroba(state: EstadoEmail) -> EstadoEmail:
    tiene_arroba = "@" in state["email"]
    return {"tiene_arroba": tiene_arroba}

# Solución: Nodo verificar dominio
def nodo_verificar_dominio(state: EstadoEmail) -> EstadoEmail:
    email = state["email"]
    dominios_validos = [".com", ".es", ".org"]
    dominio_valido = any(email.endswith(dom) for dom in dominios_validos)
    return {"dominio_valido": dominio_valido}

# Solución: Router de validación
def router_validacion(state: EstadoEmail) -> str:
    if state["tiene_arroba"] and state["dominio_valido"]:
        return "valido"
    else:
        return "invalido"

# Solución: Nodos finales
def nodo_email_valido(state: EstadoEmail) -> EstadoEmail:
    return {"resultado": "Email válido"}

def nodo_email_invalido(state: EstadoEmail) -> EstadoEmail:
    return {"resultado": "Email inválido"}

# Solución: Construir grafo
workflow_email = StateGraph(EstadoEmail)

# Añadir nodos
workflow_email.add_node("verificar_arroba", nodo_verificar_arroba)
workflow_email.add_node("verificar_dominio", nodo_verificar_dominio)
workflow_email.add_node("valido", nodo_email_valido)
workflow_email.add_node("invalido", nodo_email_invalido)

# Entry point
workflow_email.set_entry_point("verificar_arroba")

# Conectar edges
workflow_email.add_edge("verificar_arroba", "verificar_dominio")

# Edge condicional después de verificar dominio
workflow_email.add_conditional_edges(
    "verificar_dominio",
    router_validacion,
    {
        "valido": "valido",
        "invalido": "invalido"
    }
)

# Finales a END
workflow_email.add_edge("valido", END)
workflow_email.add_edge("invalido", END)

# Compilar
app_email = workflow_email.compile()

# Pruebas
emails_test = [
    "usuario@example.com",
    "invalido.com",
    "test@dominio.es",
    "sin_arroba.org"
]

for email in emails_test:
    estado = {"email": email, "tiene_arroba": False, "dominio_valido": False, "resultado": ""}
    resultado = app_email.invoke(estado)
    print(f"Email: {email} → Resultado: {resultado['resultado']}")

---
## EJERCICIO 6: Contador con Límite Configurable

In [ ]:
class EstadoContadorConfigurable(TypedDict):
    contador: int
    limite: int
    mensaje: str

# Solución: Nodo incrementador
def nodo_incrementar(state: EstadoContadorConfigurable) -> EstadoContadorConfigurable:
    nuevo_contador = state["contador"] + 1
    print(f"Incrementando: {state['contador']} → {nuevo_contador}")
    return {"contador": nuevo_contador}

# Solución: Router con lógica de ciclo
def router_limite(state: EstadoContadorConfigurable) -> str:
    if state["contador"] < state["limite"]:
        return "continuar"  # Vuelve a incrementar (ciclo)
    else:
        return "terminar"  # Sale del ciclo

# Solución: Nodo final
def nodo_completar(state: EstadoContadorConfigurable) -> EstadoContadorConfigurable:
    mensaje = f"Contador completado: alcanzó {state['contador']} de {state['limite']}"
    return {"mensaje": mensaje}

# Solución: Construir grafo con ciclo
workflow_contador = StateGraph(EstadoContadorConfigurable)

# Añadir nodos
workflow_contador.add_node("incrementar", nodo_incrementar)
workflow_contador.add_node("completar", nodo_completar)

# Entry point
workflow_contador.set_entry_point("incrementar")

# Edge condicional con CICLO
workflow_contador.add_conditional_edges(
    "incrementar",
    router_limite,
    {
        "continuar": "incrementar",  # ¡CICLO!
        "terminar": "completar"
    }
)

# Final a END
workflow_contador.add_edge("completar", END)

# Compilar
app_contador = workflow_contador.compile()

# Prueba
estado_inicial = {"contador": 0, "limite": 5, "mensaje": ""}
resultado = app_contador.invoke(estado_inicial)
print(f"\n{resultado['mensaje']}")

# PREMIO VISUAL
print("\n" + "="*60)
print("ESTRUCTURA DEL GRAFO:")
print("="*60)
print(app_contador.get_graph().draw_ascii())

---
## EJERCICIO 7: Sistema de Calificación con Retry

In [ ]:
import random

class EstadoCalificacion(TypedDict):
    calificacion: float
    intentos: int
    resultado: str

# Solución: Nodo generador de calificación
def nodo_generar_calificacion(state: EstadoCalificacion) -> EstadoCalificacion:
    intentos = state["intentos"] + 1
    calificacion = round(random.uniform(0, 10), 2)
    print(f"Intento {intentos}: Calificación = {calificacion}")
    return {"calificacion": calificacion, "intentos": intentos}

# Solución: Router con lógica de retry
def router_calificacion(state: EstadoCalificacion) -> str:
    if state["calificacion"] >= 5:
        return "aprobado"
    elif state["intentos"] >= 3:
        return "reprobado"  # Máximo de intentos
    else:
        return "reintentar"  # Ciclo

# Solución: Nodos finales
def nodo_aprobar(state: EstadoCalificacion) -> EstadoCalificacion:
    return {"resultado": "APROBADO"}

def nodo_reprobar(state: EstadoCalificacion) -> EstadoCalificacion:
    return {"resultado": "REPROBADO (máximo de intentos)"}

# Solución: Construir grafo
workflow_calificacion = StateGraph(EstadoCalificacion)

# Añadir nodos
workflow_calificacion.add_node("generar", nodo_generar_calificacion)
workflow_calificacion.add_node("aprobar", nodo_aprobar)
workflow_calificacion.add_node("reprobar", nodo_reprobar)

# Entry point
workflow_calificacion.set_entry_point("generar")

# Edge condicional con ciclo
workflow_calificacion.add_conditional_edges(
    "generar",
    router_calificacion,
    {
        "aprobado": "aprobar",
        "reprobado": "reprobar",
        "reintentar": "generar"  # ¡CICLO!
    }
)

# Finales a END
workflow_calificacion.add_edge("aprobar", END)
workflow_calificacion.add_edge("reprobar", END)

# Compilar
app_calificacion = workflow_calificacion.compile()

# Prueba
estado_inicial = {"calificacion": 0.0, "intentos": 0, "resultado": ""}
resultado = app_calificacion.invoke(estado_inicial)
print(f"\nCalificación final: {resultado['calificacion']:.1f}")
print(f"Intentos necesarios: {resultado['intentos']}")
print(f"Resultado: {resultado['resultado']}")

---
## EJERCICIO 8: Pipeline de Procesamiento de Pedidos

In [ ]:
class EstadoPedido(TypedDict):
    cantidad_pedido: int
    stock_disponible: int
    precio_unitario: float
    total: float
    estado_pedido: str

# Solución: Nodo validar stock
def nodo_validar_stock(state: EstadoPedido) -> EstadoPedido:
    # No modifica estado, solo pasa para que el router decida
    print(f"Validando stock: pedido={state['cantidad_pedido']}, disponible={state['stock_disponible']}")
    return state

# Solución: Router de stock
def router_stock(state: EstadoPedido) -> str:
    if state["cantidad_pedido"] <= state["stock_disponible"]:
        return "procesar"
    else:
        return "cancelar"

# Solución: Nodo calcular total
def nodo_calcular_total(state: EstadoPedido) -> EstadoPedido:
    total = state["cantidad_pedido"] * state["precio_unitario"]
    print(f"Calculando total: {state['cantidad_pedido']} x ${state['precio_unitario']} = ${total}")
    return {"total": total}

# Solución: Nodo aplicar descuento
def nodo_aplicar_descuento(state: EstadoPedido) -> EstadoPedido:
    total = state["total"]
    if total > 100:
        descuento = total * 0.10
        total = total - descuento
        print(f"Descuento aplicado (10%): nuevo total = ${total:.2f}")
    return {"total": total}

# Solución: Nodos finales
def nodo_confirmar(state: EstadoPedido) -> EstadoPedido:
    print("Pedido confirmado")
    return {"estado_pedido": "CONFIRMADO"}

def nodo_cancelar(state: EstadoPedido) -> EstadoPedido:
    print("Pedido cancelado por falta de stock")
    return {"estado_pedido": "CANCELADO"}

# Solución: Construir grafo
workflow_pedido = StateGraph(EstadoPedido)

# Añadir nodos
workflow_pedido.add_node("validar_stock", nodo_validar_stock)
workflow_pedido.add_node("calcular_total", nodo_calcular_total)
workflow_pedido.add_node("aplicar_descuento", nodo_aplicar_descuento)
workflow_pedido.add_node("confirmar", nodo_confirmar)
workflow_pedido.add_node("cancelar", nodo_cancelar)

# Entry point
workflow_pedido.set_entry_point("validar_stock")

# Edge condicional desde validar_stock
workflow_pedido.add_conditional_edges(
    "validar_stock",
    router_stock,
    {
        "procesar": "calcular_total",
        "cancelar": "cancelar"
    }
)

# Flujo si hay stock: calcular → descuento → confirmar
workflow_pedido.add_edge("calcular_total", "aplicar_descuento")
workflow_pedido.add_edge("aplicar_descuento", "confirmar")

# Finales a END
workflow_pedido.add_edge("confirmar", END)
workflow_pedido.add_edge("cancelar", END)

# Compilar
app_pedido = workflow_pedido.compile()

# Pruebas
pedido_con_stock = {
    "cantidad_pedido": 5,
    "stock_disponible": 10,
    "precio_unitario": 25.0,
    "total": 0.0,
    "estado_pedido": ""
}

pedido_sin_stock = {
    "cantidad_pedido": 15,
    "stock_disponible": 10,
    "precio_unitario": 25.0,
    "total": 0.0,
    "estado_pedido": ""
}

print("PEDIDO 1 (Con stock):")
resultado1 = app_pedido.invoke(pedido_con_stock)
print(f"Estado: {resultado1['estado_pedido']} - Total: ${resultado1['total']:.2f}\n")

print("PEDIDO 2 (Sin stock):")
resultado2 = app_pedido.invoke(pedido_sin_stock)
print(f"Estado: {resultado2['estado_pedido']}")

# PREMIO VISUAL
print("\n" + "="*60)
print("ESTRUCTURA DEL GRAFO:")
print("="*60)
print(app_pedido.get_graph().draw_ascii())

---
## EJERCICIO 9: Sistema de Generación de Contraseña con Validación Iterativa

In [ ]:
import string
import random

class EstadoPassword(TypedDict):
    password: str
    es_valida: bool
    intentos: int
    resultado: str

# Solución: Nodo generador
def nodo_generar_password(state: EstadoPassword) -> EstadoPassword:
    caracteres = string.ascii_letters + string.digits
    password = ''.join(random.choices(caracteres, k=8))
    intentos = state["intentos"] + 1
    print(f"Intento {intentos}: Generando password...")
    return {"password": password, "intentos": intentos}

# Solución: Nodo validador
def nodo_validar_password(state: EstadoPassword) -> EstadoPassword:
    password = state["password"]
    # Verificar al menos 1 letra Y 1 número
    tiene_letra = any(c.isalpha() for c in password)
    tiene_numero = any(c.isdigit() for c in password)
    es_valida = tiene_letra and tiene_numero
    
    print(f"  Password: {password} - Válida: {es_valida}")
    return {"es_valida": es_valida}

# Solución: Router
def router_password(state: EstadoPassword) -> str:
    if state["es_valida"]:
        return "aceptar"
    elif state["intentos"] >= 5:
        return "fallo"
    else:
        return "regenerar"  # Ciclo

# Solución: Nodos finales
def nodo_aceptar_password(state: EstadoPassword) -> EstadoPassword:
    return {"resultado": "Password aceptada"}

def nodo_fallo_password(state: EstadoPassword) -> EstadoPassword:
    return {"resultado": "Fallo: máximo de intentos alcanzado"}

# Solución: Construir grafo
workflow_password = StateGraph(EstadoPassword)

# Añadir nodos
workflow_password.add_node("generar", nodo_generar_password)
workflow_password.add_node("validar", nodo_validar_password)
workflow_password.add_node("aceptar", nodo_aceptar_password)
workflow_password.add_node("fallo", nodo_fallo_password)

# Entry point
workflow_password.set_entry_point("generar")

# Flujo: generar → validar
workflow_password.add_edge("generar", "validar")

# Edge condicional desde validar
workflow_password.add_conditional_edges(
    "validar",
    router_password,
    {
        "aceptar": "aceptar",
        "fallo": "fallo",
        "regenerar": "generar"  # ¡CICLO!
    }
)

# Finales a END
workflow_password.add_edge("aceptar", END)
workflow_password.add_edge("fallo", END)

# Compilar
app_password = workflow_password.compile()

# Prueba
estado_inicial = {"password": "", "es_valida": False, "intentos": 0, "resultado": ""}
resultado = app_password.invoke(estado_inicial)
print(f"\nPassword generada: {resultado['password']}")
print(f"Intentos necesarios: {resultado['intentos']}")
print(f"Resultado: {resultado['resultado']}")

---
## EJERCICIO 10: Sistema Multi-Etapa con Múltiples Bifurcaciones

In [ ]:
class EstadoCredito(TypedDict):
    edad: int
    ingresos: float
    monto_solicitado: float
    score_riesgo: float
    tasa_interes: float
    estado_solicitud: str
    razon_rechazo: str

# ETAPA 1: Verificación de edad
def nodo_verificar_edad(state: EstadoCredito) -> EstadoCredito:
    print(f"Verificando edad: {state['edad']} años")
    return state

def router_edad(state: EstadoCredito) -> str:
    if state["edad"] < 18:
        return "rechazar_edad"
    else:
        return "verificar_ingresos"

# ETAPA 2: Verificación de ingresos
def nodo_verificar_ingresos(state: EstadoCredito) -> EstadoCredito:
    print(f"Verificando ingresos: ${state['ingresos']} vs mínimo ${state['monto_solicitado']/2}")
    return state

def router_ingresos(state: EstadoCredito) -> str:
    if state["ingresos"] >= state["monto_solicitado"] / 2:
        return "calcular_riesgo"
    else:
        return "rechazar_ingresos"

# ETAPA 3: Cálculo de riesgo
def nodo_calcular_riesgo(state: EstadoCredito) -> EstadoCredito:
    score = (state["ingresos"] / state["monto_solicitado"]) * 100
    print(f"Score de riesgo calculado: {score:.2f}")
    return {"score_riesgo": score}

def router_riesgo(state: EstadoCredito) -> str:
    score = state["score_riesgo"]
    if score >= 200:
        return "aprobar_premium"
    elif score >= 100:
        return "aprobar_standard"
    else:
        return "rechazar_riesgo"

# NODOS FINALES: Aprobaciones
def nodo_aprobar_premium(state: EstadoCredito) -> EstadoCredito:
    print("✅ Aprobado: Categoría PREMIUM")
    return {
        "tasa_interes": 5.0,
        "estado_solicitud": "APROBADO",
        "razon_rechazo": ""
    }

def nodo_aprobar_standard(state: EstadoCredito) -> EstadoCredito:
    print("✅ Aprobado: Categoría STANDARD")
    return {
        "tasa_interes": 10.0,
        "estado_solicitud": "APROBADO",
        "razon_rechazo": ""
    }

# NODOS FINALES: Rechazos
def nodo_rechazar_edad(state: EstadoCredito) -> EstadoCredito:
    print("❌ Rechazado: Menor de edad")
    return {
        "estado_solicitud": "RECHAZADO",
        "razon_rechazo": "Debe ser mayor de 18 años"
    }

def nodo_rechazar_ingresos(state: EstadoCredito) -> EstadoCredito:
    print("❌ Rechazado: Ingresos insuficientes")
    return {
        "estado_solicitud": "RECHAZADO",
        "razon_rechazo": "Ingresos deben ser al menos 50% del monto solicitado"
    }

def nodo_rechazar_riesgo(state: EstadoCredito) -> EstadoCredito:
    print("❌ Rechazado: Score de riesgo bajo")
    return {
        "estado_solicitud": "RECHAZADO",
        "razon_rechazo": "Score de riesgo insuficiente"
    }

# SOLUCIÓN: Construir grafo multi-etapa
workflow_credito = StateGraph(EstadoCredito)

# Añadir todos los nodos
workflow_credito.add_node("verificar_edad", nodo_verificar_edad)
workflow_credito.add_node("verificar_ingresos", nodo_verificar_ingresos)
workflow_credito.add_node("calcular_riesgo", nodo_calcular_riesgo)
workflow_credito.add_node("aprobar_premium", nodo_aprobar_premium)
workflow_credito.add_node("aprobar_standard", nodo_aprobar_standard)
workflow_credito.add_node("rechazar_edad", nodo_rechazar_edad)
workflow_credito.add_node("rechazar_ingresos", nodo_rechazar_ingresos)
workflow_credito.add_node("rechazar_riesgo", nodo_rechazar_riesgo)

# Entry point
workflow_credito.set_entry_point("verificar_edad")

# ETAPA 1: Router de edad
workflow_credito.add_conditional_edges(
    "verificar_edad",
    router_edad,
    {
        "rechazar_edad": "rechazar_edad",
        "verificar_ingresos": "verificar_ingresos"
    }
)

# ETAPA 2: Router de ingresos
workflow_credito.add_conditional_edges(
    "verificar_ingresos",
    router_ingresos,
    {
        "rechazar_ingresos": "rechazar_ingresos",
        "calcular_riesgo": "calcular_riesgo"
    }
)

# ETAPA 3: Router de riesgo
workflow_credito.add_conditional_edges(
    "calcular_riesgo",
    router_riesgo,
    {
        "aprobar_premium": "aprobar_premium",
        "aprobar_standard": "aprobar_standard",
        "rechazar_riesgo": "rechazar_riesgo"
    }
)

# Todos los nodos finales van a END
workflow_credito.add_edge("aprobar_premium", END)
workflow_credito.add_edge("aprobar_standard", END)
workflow_credito.add_edge("rechazar_edad", END)
workflow_credito.add_edge("rechazar_ingresos", END)
workflow_credito.add_edge("rechazar_riesgo", END)

# Compilar
app_credito = workflow_credito.compile()

# Pruebas
casos_prueba = [
    {
        "edad": 25,
        "ingresos": 3000.0,
        "monto_solicitado": 1000.0,
        "score_riesgo": 0.0,
        "tasa_interes": 0.0,
        "estado_solicitud": "",
        "razon_rechazo": ""
    },
    {
        "edad": 30,
        "ingresos": 1500.0,
        "monto_solicitado": 1000.0,
        "score_riesgo": 0.0,
        "tasa_interes": 0.0,
        "estado_solicitud": "",
        "razon_rechazo": ""
    },
    {
        "edad": 22,
        "ingresos": 800.0,
        "monto_solicitado": 1000.0,
        "score_riesgo": 0.0,
        "tasa_interes": 0.0,
        "estado_solicitud": "",
        "razon_rechazo": ""
    },
    {
        "edad": 17,
        "ingresos": 5000.0,
        "monto_solicitado": 1000.0,
        "score_riesgo": 0.0,
        "tasa_interes": 0.0,
        "estado_solicitud": "",
        "razon_rechazo": ""
    }
]

for i, caso in enumerate(casos_prueba, 1):
    print(f"\nCASO {i}: Edad={caso['edad']}, Ingresos=${caso['ingresos']}, Solicita=${caso['monto_solicitado']}")
    resultado = app_credito.invoke(caso)
    print(f"Resultado: {resultado['estado_solicitud']}")
    if resultado['estado_solicitud'] == "APROBADO":
        print(f"Tasa de interés: {resultado['tasa_interes']}%")
    else:
        print(f"Razón: {resultado['razon_rechazo']}")

# PREMIO VISUAL
print("\n" + "="*60)
print("ESTRUCTURA DEL GRAFO (Sistema de Crédito):")
print("="*60)
print(app_credito.get_graph().draw_ascii())